# Imports

In [1]:
%pip install serpapi
%pip install google-search-results

from serpapi import GoogleSearch
import pickle
import pandas as pd
import re

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Requesting data

In [2]:
def get_comments(query : str):

    api_key = pickle.load(open("key.pkl", "rb"))
    
    params = {
    "q": query,
    "location": "Netherlands",
    "hl": "en",
    "gl": "nl",
    "google_domain": "google.nl",
    "api_key": api_key,
    "start": 0,
    "num": 100000
    }

    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        print("Request successful")
    except:
        print("Request failed")

    return results

search_results = get_comments('"hittegolf" and "nederland" after:2013-05-01 before:2023-09-30 site:https://www.nu.nl')
print(search_results)

Request successful
{'search_metadata': {'id': '659b22a3ea14429394d17154', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/7c71119f19c8b1a4/659b22a3ea14429394d17154.json', 'created_at': '2024-01-07 22:16:03 UTC', 'processed_at': '2024-01-07 22:16:03 UTC', 'google_url': 'https://www.google.nl/search?q=%22nationaal+hitteplan%22+after%3A2013-05-01+before%3A2023-09-30+site%3Ahttps%3A%2F%2Fwww.nu.nl&oq=%22nationaal+hitteplan%22+after%3A2013-05-01+before%3A2023-09-30+site%3Ahttps%3A%2F%2Fwww.nu.nl&uule=w+CAIQICILTmV0aGVybGFuZHM&hl=en&gl=nl&num=100000&start=0&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/7c71119f19c8b1a4/659b22a3ea14429394d17154.html', 'total_time_taken': 1.79}, 'search_parameters': {'engine': 'google', 'q': '"nationaal hitteplan" after:2013-05-01 before:2023-09-30 site:https://www.nu.nl', 'location_requested': 'Netherlands', 'location_used': 'Netherlands', 'google_domain': 'google.nl', 'hl': 'en', 'gl': 'nl', 'start': 0, 'num': 

In [3]:
# Saving the raw data since we have limited API requests
pickle.dump(search_results, open("api_results.pkl", "wb"))

# Processing data

In [5]:
search_results = pickle.load(open("api_results.pkl", "rb"))

# getting the actual google search results from returned dataset and converting to a df
results = search_results["organic_results"]
df = pd.DataFrame.from_dict(results)

# adding the subpages in the URL as "category" tag
df["category"] = df["displayed_link"].apply(lambda x: re.sub(r"https://www\.nu\.nl\s*›\s*", "", x))
# The old page for "Actueel" was "dvn" or "dvn2"
df["category"] = df["category"].apply(lambda x: "Actueel" if re.search(r"dvn", x) else x)
# removing text beyond the first nu.nl subpage from categories, then capitalising 
df["category"] = df["category"].apply(lambda x: re.sub(r' ›.*$', '', x).capitalize())
# converting the date column to datetime format
df["date"] = df["date"].apply(lambda x: pd.to_datetime(x, format="mixed"))
# dropping "unneccesary" columns
# df = df.drop(["displayed_link", "favicon", "snippet_highlighted_words", "about_page_link", \
#               "about_page_serpapi_link", "cached_page_link"], axis="columns")
df = df.drop(["displayed_link", "favicon", "snippet_highlighted_words"], axis="columns")

# dropping irrelevant rows based on their category
df = df.drop(df[df.category.isin(["Advertorial-nunl", "2013", "Recensies", "Autoscout24"])].index)

df = df.rename({"link": "url"}, axis="columns")
df = df.rename({"position": "search_rank"}, axis="columns")

# Adding empty columns for scraped data, and a scraped status
df["scraped"] = False
df["article_text"] = ""
df["subheadings"] = ""
df["tag"] = ""
df["img_urls"] = ""
df["comments_present"] = ""

df = df.reindex(columns=["search_rank", "title", "source", "date", "category", "url", "snippet", \
                         "article_text", "subheadings", "tag", "img_urls", "comments_present", "scraped"])
# sorting by date and resetting index
df = df.sort_values(by="date")
df = df.reset_index(drop=True)
display(df)

,search_rank,title,source,date,category,url,snippet,article_text,subheadings,tag,img_urls,comments_present,scraped
0,13,2014: 'Zomerse dag eindigt in alle opzichten g...,NU,2014-12-22,2014-zomerse...,https://www.nu.nl/jaaroverzicht2014/3954994/20...,"Het is warm, veel mensen hebben vakantie en KN...",,,,,,False
1,11,Het weer in juli: maand met extreem weer,NU,2015-07-29,Actueel,https://www.nu.nl/dvn/4097103/weer-in-juli-maa...,"Het Nationaal Hitteplan, dat gezondheidsproble...",,,,,,False
2,9,"Sproeiverbod voor delen van Zuid-Holland, Utre...",NU,2018-07-23,Binnenland,https://www.nu.nl/binnenland/5379440/sproeiver...,Eerder · Achtergrond |'Voor boeren kan schade ...,,,,,,False
3,12,Warmterecord gebroken op vijftien meetplekken ...,NU,2018-07-26,Binnenland,https://www.nu.nl/binnenland/5385326/warmterec...,Het RIVM stelde begin deze week het Nationaal ...,,,,,,False
4,10,Op deze plekken in en rondom Utrecht kan je zw...,NU,2019-07-25,Utrecht,https://www.nu.nl/utrecht/5969089/op-deze-plek...,Voor de komende dagen heeft het KNMI code oran...,,,,,,False
5,2,Sporten in de hitte: Doen of niet doen? | Binn...,NU,2019-08-26,Binnenland,https://www.nu.nl/binnenland/5983561/sporten-i...,Donderdag 8 augustus: De zon kleurt het landsc...,,,,,,False
6,14,'Popcornbuien' oorzaak van plaatselijk noodwee...,NU,2019-08-28,Binnenland,https://www.nu.nl/binnenland/5984392/popcornbu...,"Nationaal Hitteplan van kracht, 'waarschijnlij...",,,,,,False
7,8,Regen- en onweersbuien trekken over het land |...,NU,2020-08-06,Binnenland,https://www.nu.nl/binnenland/6069124/regen-en-...,Nationaal Hitteplan is actief. Het Nationaal H...,,,,,,False
8,7,Zwemmen in de stad: waar wel én waar juist niet?,NU,2022-07-18,Amsterdam,https://www.nu.nl/amsterdam/6212708/zwemmen-in...,Vanaf maandag treedt het Nationaal Hitteplan i...,,,,,,False
9,4,Onze steeds hetere zomers in zes grafieken | N...,NU,2022-07-19,Nu-klimaat,https://www.nu.nl/nu-klimaat/6212830/onze-stee...,Vier hotspots van klimaatverandering die de to...,,,,,,False


In [6]:
pickle.dump(df, open("api_results_processed.df.pkl", "wb"))

### Initialising and pickling an empty DataFrame for comments

In [7]:
df_comments_initial = pd.DataFrame(columns=["url", "username", "expertise", "timestamp", "comment_text", "likes"])
df_comments_initial.to_pickle("all_comments.df.pkl")

In [8]:
display(df)
display(df_comments_initial)

,search_rank,title,source,date,category,url,snippet,article_text,subheadings,tag,img_urls,comments_present,scraped
0,13,2014: 'Zomerse dag eindigt in alle opzichten g...,NU,2014-12-22,2014-zomerse...,https://www.nu.nl/jaaroverzicht2014/3954994/20...,"Het is warm, veel mensen hebben vakantie en KN...",,,,,,False
1,11,Het weer in juli: maand met extreem weer,NU,2015-07-29,Actueel,https://www.nu.nl/dvn/4097103/weer-in-juli-maa...,"Het Nationaal Hitteplan, dat gezondheidsproble...",,,,,,False
2,9,"Sproeiverbod voor delen van Zuid-Holland, Utre...",NU,2018-07-23,Binnenland,https://www.nu.nl/binnenland/5379440/sproeiver...,Eerder · Achtergrond |'Voor boeren kan schade ...,,,,,,False
3,12,Warmterecord gebroken op vijftien meetplekken ...,NU,2018-07-26,Binnenland,https://www.nu.nl/binnenland/5385326/warmterec...,Het RIVM stelde begin deze week het Nationaal ...,,,,,,False
4,10,Op deze plekken in en rondom Utrecht kan je zw...,NU,2019-07-25,Utrecht,https://www.nu.nl/utrecht/5969089/op-deze-plek...,Voor de komende dagen heeft het KNMI code oran...,,,,,,False
5,2,Sporten in de hitte: Doen of niet doen? | Binn...,NU,2019-08-26,Binnenland,https://www.nu.nl/binnenland/5983561/sporten-i...,Donderdag 8 augustus: De zon kleurt het landsc...,,,,,,False
6,14,'Popcornbuien' oorzaak van plaatselijk noodwee...,NU,2019-08-28,Binnenland,https://www.nu.nl/binnenland/5984392/popcornbu...,"Nationaal Hitteplan van kracht, 'waarschijnlij...",,,,,,False
7,8,Regen- en onweersbuien trekken over het land |...,NU,2020-08-06,Binnenland,https://www.nu.nl/binnenland/6069124/regen-en-...,Nationaal Hitteplan is actief. Het Nationaal H...,,,,,,False
8,7,Zwemmen in de stad: waar wel én waar juist niet?,NU,2022-07-18,Amsterdam,https://www.nu.nl/amsterdam/6212708/zwemmen-in...,Vanaf maandag treedt het Nationaal Hitteplan i...,,,,,,False
9,4,Onze steeds hetere zomers in zes grafieken | N...,NU,2022-07-19,Nu-klimaat,https://www.nu.nl/nu-klimaat/6212830/onze-stee...,Vier hotspots van klimaatverandering die de to...,,,,,,False


,url,username,expertise,timestamp,comment_text,likes
